# Google AutoML Vision sobre el conjunto de datos MNIST

In [73]:
import os
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from google.cloud import automl
from google.cloud import storage 

In [75]:
project_id = "clasificador-mnis-automl"
location = "us-central1"

## Sobre el conjunto de datos de entrenamiento

Google AutoML Vision acepta imágenes en los siguientes formatos:

* JPEG
* PNG
* GIF
* BMP
* ICO

En un tamaño de hasta 30MB

Por lo tanto usamos el conjunto de datos MNIST convertidos a jpg usando: [MNIST-JPG](https://github.com/teavanist/MNIST-JPG)

Se usaron alrededor de 4500 imágenes por cada etiqueta (0..9)

## Ingesta de los datos

La ingesta de datos en Google AutoML Vision se puede hacer de dos formas:

* Subiendo un archivo comprimido con las imágenes agrupadas en directorios donde cada directorio es una etiqueta correspondiente a las imágenes que contiene
* Subiendo las imágenes a Google Cloud Storage y un archivo csv que contiene tres campos la partición destino (TRAIN | TEST | VALIDATION), la ruta de la imagen en Storage y la etiqueta (o etiquetas en caso que corresponda)

Elegimos la segunda alternativa que permite realizar la partición de los conjuntos dentro del mismo archivo


### Upload de imágenes 

Usamos la interfaz gráfica para hacer el upload de las imágenes


<img src="imagenes/cloud_storage_imagenes.png">


### Archivo csv

Usamos un script de bash para generar el archivo csv de etiquetas:

    #!/bin/bash
    for i in 0 1 2 3 4 5 6 7 8 9
        do
                for f in $(gsutil ls gs://clasificador-mnis-automl-vcm/mnist/trainingSet/$i);
                        do echo TRAIN,$f,$i;
                done
    done >> labels_train.csv


El archivo resultante de este proceso se mira así: 

    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_1.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10007.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10010.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10017.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10032.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10039.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10043.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10059.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10074.jpg,0
    TEST,gs://clasificador-mnis-automl-vcm/mnist/trainingSet/0/img_10082.jpg,0
    ...


y subimos el archivo resultante al storage usando la herramienta de línea de comandos de Cloud Storage


    gsutil cp  labels_train.csv  gs://clasificador-mnis-automl-vcm/mnist/



In [63]:
path_train = "gs://clasificador-mnis-automl-vcm/mnist/labels_train.csv"
df_train = pd.read_csv(path_train, header=None)
df_train

,0,1,2
0,TEST,gs://clasificador-mnis-automl-vcm/mnist/traini...,0
1,TEST,gs://clasificador-mnis-automl-vcm/mnist/traini...,0
2,TEST,gs://clasificador-mnis-automl-vcm/mnist/traini...,0
3,TEST,gs://clasificador-mnis-automl-vcm/mnist/traini...,0
4,TEST,gs://clasificador-mnis-automl-vcm/mnist/traini...,0
...,...,...,...
40086,TRAIN,gs://clasificador-mnis-automl-vcm/mnist/traini...,9
40087,TRAIN,gs://clasificador-mnis-automl-vcm/mnist/traini...,9
40088,TRAIN,gs://clasificador-mnis-automl-vcm/mnist/traini...,9
40089,TRAIN,gs://clasificador-mnis-automl-vcm/mnist/traini...,9


## Crear el conjunto de datos 

In [80]:
client = automl.AutoMlClient()
project_location = f"projects/{project_id}/locations/{location}"

# Especificar el tipo de clasificación:
# MultiLabel: Multiple labels are allowed for one example.
# MultiClass: At most one label is allowed per example.

metadata = automl.ImageClassificationDatasetMetadata(
    classification_type=automl.ClassificationType.MULTICLASS
)
dataset = automl.Dataset(
    display_name=display_name,
    image_classification_dataset_metadata=metadata,
)

# Crear el dataset
response = client.create_dataset(parent=project_location, dataset=dataset)

created_dataset = response.result()

dataset_name = created_dataset.name
dataset_id = created_dataset.name.split("/")[-1]

# Mostrar la información del dataset creado
print("Dataset name: {}".format(dataset_name))
print("Dataset id: {}".format(dataset_id))

Dataset name: projects/739558555147/locations/us-central1/datasets/ICN6322465088351502336
Dataset id: ICN6322465088351502336


## Importar las imágenes al conjunto de datos

In [81]:
dataset_full_id = client.dataset_path(project_id, "us-central1", dataset_id)
# Obtener las URIs de Google Cloud Storage
input_uris = path_train.split(",")
gcs_source = automl.GcsSource(input_uris=input_uris)
input_config = automl.InputConfig(gcs_source=gcs_source)
# Importar los datos de la input URI
response = client.import_data(name=dataset_full_id, input_config=input_config)

print("Importando datos...")
print("Datos importados en el dataset. {}".format(response.result()))

Importando datos...
Datos importados en el dataset. 


## Entrenar el modelo

In [82]:
# el presupuesto asignado al entrenamiento será menor o igual al valor asignado
metadata = automl.ImageClassificationModelMetadata(
    train_budget_milli_node_hours=24000
)
# parámetros del modelo
model = automl.Model(
    display_name=display_name,
    dataset_id=dataset_id,
    image_classification_model_metadata=metadata,
)

# Crear (entrenar) el modelo
response = client.create_model(parent=project_location, model=model)

print("Nombre de la operación de entrenamiento: {}".format(response.operation.name))
print("Entrenando el modelo...")

Nombre de la operación de entrenamiento: projects/739558555147/locations/us-central1/operations/ICN2376102077106487296
Entrenando el modelo...


<img src="imagenes/train_model.png" />
<img src="imagenes/model_created.png" />

## Evaluación del modelo obtenido

El proceso de entrenamiento para aproximadamente 40000 imágenes duró alrededor de 120 minutos.

El valor de la precisión promedio fue de 0.998. Para un intervalo de confianza de 0.5 l

En la matriz de confusión se puede observar que la performance fue muy buena para todas las clases, con un desempeño levemente inferior en el caso de las clases 3 y 5

<img src="imagenes/evaluation_03.png" />
<img src="imagenes/evaluation_01.png" />
<img src="imagenes/evaluation_02.png" />

In [88]:
model_id = "ICN3874320535491969024"

client = automl.AutoMlClient()
model_full_id = client.model_path(project_id, "us-central1", model_id)

print("Lista de evaluaciones del modelo:")
for evaluation in client.list_model_evaluations(parent=model_full_id, filter=""):
    print("Nombre de la evaluación: {}".format(evaluation.name))
    print(
        "Métricas de evaluación del modelo de clasificación: {}".format(
            evaluation.classification_evaluation_metrics
        )
    )

Lista de evaluaciones del modelo:
Nombre de la evaluación: projects/739558555147/locations/us-central1/models/ICN3874320535491969024/modelEvaluations/2171323628263743564
Métricas de evaluación del modelo de clasificación: au_prc: 0.9994238018989563
confidence_metrics_entry {
  recall: 1.0
  precision: 0.10000000149011612
}
confidence_metrics_entry {
  confidence_threshold: 0.05000000074505806
  recall: 1.0
  precision: 0.8928571343421936
}
confidence_metrics_entry {
  confidence_threshold: 0.10000000149011612
  recall: 1.0
  precision: 0.9345794320106506
}
confidence_metrics_entry {
  confidence_threshold: 0.15000000596046448
  recall: 1.0
  precision: 0.9345794320106506
}
confidence_metrics_entry {
  confidence_threshold: 0.20000000298023224
  recall: 0.9900000095367432
  precision: 0.9611650705337524
}
confidence_metrics_entry {
  confidence_threshold: 0.25
  recall: 0.9900000095367432
  precision: 0.9611650705337524
}
confidence_metrics_entry {
  confidence_threshold: 0.300000011920